In [1]:
%run ../../src/start.py


python	3.9.5 (default, Jun  4 2021, 12:28:51) 
[GCC 7.5.0]
---------------------
Versions:
----------------------
pandas      1.3.0
numpy       1.20.2
matplotlib  3.3.4
seaborn     0.11.1
plotly      5.1.0
boto3       1.17.109
awswrangler 2.10.0
----------------------


Loaded Libraries
-------------------
import pandas as pd
import numpy as np
import sys,os
import re
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


## AWS
import boto3
import awswrangler as wr
----------------


GLOBAL VARIABLES
--------------------------
# Creating the low level functional client
client = boto3.client(
    's3',
    aws_access_key_id = AWS_KEY_ID,
    aws_secret_access_key = AWS_SECRET,
)
    
# Creating the high level object oriented interface
resource = boto3.resource(
    's3',
    aws_access_key_id = AWS_KEY_

## Load Data

In [2]:
# year file
year = '2014'

In [3]:
# helper function
def bucket_raw_path(bucket_name,path_dir):
    ''''''
    raw_path = f's3://{bucket_name}/{path_dir}'
    return raw_path

In [4]:
raw = bucket_raw_path('dend-data',f'capstone/raw-data/crime-data/{year}')
raw

's3://dend-data/capstone/raw-data/crime-data/2014'

## List objects in bucket

In [5]:
wr.s3.list_objects(raw)

['s3://dend-data/capstone/raw-data/crime-data/2014/apr14.xlsx',
 's3://dend-data/capstone/raw-data/crime-data/2014/aug14.xlsx',
 's3://dend-data/capstone/raw-data/crime-data/2014/dec14.xlsx',
 's3://dend-data/capstone/raw-data/crime-data/2014/feb14.xlsx',
 's3://dend-data/capstone/raw-data/crime-data/2014/jan14.xlsx',
 's3://dend-data/capstone/raw-data/crime-data/2014/jul14.xlsx',
 's3://dend-data/capstone/raw-data/crime-data/2014/jun14.xlsx',
 's3://dend-data/capstone/raw-data/crime-data/2014/mar14.xlsx',
 's3://dend-data/capstone/raw-data/crime-data/2014/may14.xlsx',
 's3://dend-data/capstone/raw-data/crime-data/2014/nov14.xlsx',
 's3://dend-data/capstone/raw-data/crime-data/2014/oct14.xlsx',
 's3://dend-data/capstone/raw-data/crime-data/2014/sep14.xlsx']

## Create dataframe from s3 files

In [6]:
def s3_files_to_df(s3_files_path):
    ''' load s3 file path from wr result
    returns a dataframe of concat files '''
    df_list = []
    for i in wr.s3.list_objects(s3_files_path):
        temp = wr.s3.read_excel(i, engine='openpyxl', dtype={'Hour': str})  # added dtype error fix for Hour
        df_list.append(temp)
    # create df from list files
    df = pd.concat(df_list, ignore_index=True)
    return df

In [7]:
%%time
df = s3_files_to_df(raw)

CPU times: user 18 s, sys: 38.3 ms, total: 18.1 s
Wall time: 27.5 s


In [ ]:
## cleanup columns
#df.columns = df.columns.str.lower().str.replace(' ','_').str.replace('\n','_').str.strip()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121750 entries, 0 to 121749
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           121746 non-null  datetime64[ns]
 1   Hour           121750 non-null  object        
 2   Offense Type   121750 non-null  object        
 3   Beat           121734 non-null  object        
 4   Premise        92169 non-null   object        
 5   Block Range    69989 non-null   object        
 6   Street Name    70146 non-null   object        
 7   Type           121750 non-null  object        
 8   Suffix         121750 non-null  object        
 9   . Of Offenses  52214 non-null   float64       
 10  BlockRange     51604 non-null   object        
 11  StreetName     51604 non-null   object        
 12  . offenses     51604 non-null   float64       
 13  Field11        0 non-null       float64       
 14  . Offenses     10319 non-null   float64       
 15  

In [10]:
df.head()

,Date,Hour,Offense Type,Beat,Premise,Block Range,Street Name,Type,Suffix,. Of Offenses,BlockRange,StreetName,. offenses,Field11,. Offenses,. Of
0,2014-04-19,17,Murder,14D30,18D,8500-8599,MARTIN LUTHER KING,BLVD,-,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-04-28,05,Murder,10H20,13R,3600-3699,MCKINNEY,ST,-,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-04-27,18,Murder,5F30,20A,7400-7499,HILLMONT,-,-,3.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-04-09,18,Murder,17E10,20A,5400-5499,RENWICK,-,-,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-04-24,03,Murder,18F50,03B,9300-9399,RICHMOND,AVE,-,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.Hour.value_counts(dropna=False)

15     7025
18     7004
16     6953
12     6792
17     6785
14     6593
19     6467
13     6263
20     6155
10     6104
11     6069
09     5718
21     5512
08     5281
22     5231
00     4621
07     4279
23     4257
06     2857
02     2558
01     2535
03     2027
05     1954
04     1709
'00      91
'12      73
'18      69
'22      64
'17      56
'16      53
'20      51
'23      48
'21      48
'19      48
'14      44
'15      43
'08      43
'10      40
'13      38
'11      33
'07      29
'09      25
'02      25
'01      24
'06      17
'05      16
'03      12
'04      10
24        1
Name: Hour, dtype: int64

In [8]:
print(f'unique hours:{len(df.Hour.unique())}')


unique hours:49


## Show null data

In [ ]:
df.apply(lambda x: sum(x.isnull()))

In [ ]:
# create null percentage of column as dataframe
null_per_df = pd.DataFrame(df.isna().mean().round(4) * 100)
# rename columns
null_per_df.rename(columns={'index': 'col_name', 0: 'null_percent'}, inplace=True)

# filter data <20% null values
per_filter = null_per_df.null_percent<=90
null_per_df[per_filter].sort_values(by='null_percent',ascending=False)

In [ ]:
plt.figure(figsize=(15,4))
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')
plt.title('Missing data')
plt.show()

## Change Hour column errors

In [ ]:
df['Hour'].unique()

In [ ]:
len(df['Hour'].unique())

In [ ]:
df['Hour'] =  df['Hour'].str.replace('\'', '')

# TODO
# replace 00 to 24
df['Hour'].unique()

In [ ]:
len(df['Hour'].unique())

In [ ]:
# Change hour to int
df['Hour'] = df['Hour'].astype('int64')

## Change column names and drop unused

In [ ]:
df['block_range'] = pd.concat([df['Block Range'].dropna(),
                              df['BlockRange'].dropna()]).reindex_like(df)


df['street_name'] = pd.concat([df['Street Name'].dropna(),
                              df['StreetName'].dropna()]).reindex_like(df)


df['offenses'] = pd.concat([df['. Of Offenses'].dropna(),
                              df['. offenses'].dropna(),
                              df['. Offenses'].dropna(),
                              df['. Of'].dropna()]).reindex_like(df)


# drop unused cols
df = df.drop([  'Block Range','BlockRange','Street Name','StreetName','. Of Offenses','. offenses','. Offenses','. Of','Field11',
], axis = 1)






In [ ]:
## cleanup col names
df.columns = df.columns.str.lower().str.replace(' ','_').str.replace('\n','_').str.strip()

In [ ]:
df.apply(lambda x: sum(x.isnull()))

In [ ]:
plt.figure(figsize=(15,4))
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')
plt.title('Missing data')
plt.show()

In [ ]:
df[df.date.isna()]

## Drop cols with null date

In [ ]:
# remove values with nat date
df = df[~df.date.isna()].reset_index(drop=True)

In [ ]:
df[df.premise.isna()]

In [ ]:
## Change null values to UNK

df["premise"].fillna("UNK", inplace = True)

In [ ]:
df[df.block_range.isna()]

In [ ]:
## Change null values to UNK

df["block_range"].fillna("UNK", inplace = True)

In [ ]:
df[df.beat.isna()]

In [ ]:
## Change null values to UNK

df["beat"].fillna("UNK", inplace = True)

In [ ]:
df.apply(lambda x: sum(x.isnull()))

In [ ]:
plt.figure(figsize=(15,4))
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')
plt.title('Missing data')
plt.show()

## Combine columns 

## Combine date & hour to one column

In [ ]:
# convert two columns into one datetime
df['date_time'] = pd.to_datetime(df['date']) + pd.to_timedelta(df['hour'], unit='h')

# # drop columns
# df = df.drop(['date','hour'], axis = 1)

In [ ]:
df.info()

In [ ]:
df.head()

## Convert offenses to integer

In [ ]:
# convert offenses to int
df['offenses'] = df['offenses'].astype('int64')

In [ ]:
df.info()

# Create year Column

In [ ]:
df['year'] = 2014
df.head()

## change order of columns

In [ ]:
col_ord =['date_time','offenses','offense_type', 'block_range', 'street_name', 'type', 'suffix','beat', 'premise','date','hour','year']
df = df[col_ord]
df.head()

In [ ]:
df.info()

In [ ]:
df.loc[df.duplicated(keep='first'),:]

In [ ]:
df.drop_duplicates(keep='first',inplace=True)

In [ ]:
df.info()

## Save to bucket

In [ ]:
bucket = 'dend-data'
file_name = f'crime_{year}.csv'
path_to_save = f"s3://{bucket}/capstone/inter-data/crime-data/{file_name}"
path_to_save

In [ ]:
%%time
wr.s3.to_csv(df, path_to_save, index=False)

## Save Sample

In [ ]:
%%time
sample_100 = df.sample(100)


bucket = 'dend-data'
file_name = f'crime_{year}.csv'
path_to_save = f"s3://{bucket}/capstone/sample-data/crime-data/{file_name}"

wr.s3.to_csv(sample_100, path_to_save, index=False)